In [3]:
import secrets
import hashlib
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.padding import PKCS7
from cryptography.hazmat.backends import default_backend
from pydub import AudioSegment
import io

# AES Encryption and Decryption
def aes_encrypt_decrypt_audio(audio_data, key, encrypt=True):
    backend = default_backend()
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=backend)
    padder = PKCS7(algorithms.AES.block_size).padder()
    unpadder = PKCS7(algorithms.AES.block_size).unpadder()

    if encrypt:
        padded_data = padder.update(audio_data) + padder.finalize()
        encryptor = cipher.encryptor()
        return encryptor.update(padded_data) + encryptor.finalize()
    else:
        decryptor = cipher.decryptor()
        decrypted_data = decryptor.update(audio_data) + decryptor.finalize()
        return unpadder.update(decrypted_data) + unpadder.finalize()

# Encrypt the Audio File
def encrypt_audio():
    try:
        # Input prime number and generator
        p = int(input("Enter a large prime number (p): "))
        g = int(input("Enter a generator (g): "))

        # Generate private and public keys for the sender
        private_key_sender = secrets.randbelow(p - 1)
        public_key_sender = pow(g, private_key_sender, p)
        print(f"Your public key (share this with the receiver): {public_key_sender}")

        # Load the audio file
        input_audio_path = input("Enter the path of the audio file to encrypt: ")
        audio = AudioSegment.from_file(input_audio_path)
        audio_bytes = io.BytesIO()
        audio.export(audio_bytes, format="wav")
        audio_data = audio_bytes.getvalue()

        # Derive AES key from shared secret
        shared_secret = pow(g, private_key_sender, p)
        shared_secret_bytes = str(shared_secret).encode()
        aes_key = hashlib.sha256(shared_secret_bytes).digest()

        # Encrypt the audio
        encrypted_data = aes_encrypt_decrypt_audio(audio_data, aes_key, encrypt=True)

        # Save the encrypted audio
        encrypted_audio_path = "encrypted_audio.dat"
        with open(encrypted_audio_path, "wb") as f:
            f.write(encrypted_data)
        print(f"Audio encrypted and saved to {encrypted_audio_path}")

    except Exception as e:
        print(f"Error during encryption: {e}")

# Decrypt the Audio File
def decrypt_audio():
    try:
        # Input sender's public key
        public_key_sender = int(input("Enter the sender's public key: "))
        p = int(input("Enter the shared prime number (p): "))
        g = int(input("Enter the generator (g): "))

        # Generate private key for the receiver
        private_key_receiver = secrets.randbelow(p - 1)
        public_key_receiver = pow(g, private_key_receiver, p)
        print(f"Receiver's public key: {public_key_receiver}")

        # Compute the shared secret
        shared_secret = pow(public_key_sender, private_key_receiver, p)
        shared_secret_bytes = str(shared_secret).encode()
        aes_key = hashlib.sha256(shared_secret_bytes).digest()

        # Read the encrypted file
        encrypted_audio_path = input("Enter the path of the encrypted audio file: ")
        with open(encrypted_audio_path, "rb") as f:
            encrypted_data = f.read()

        # Decrypt the audio
        decrypted_data = aes_encrypt_decrypt_audio(encrypted_data, aes_key, encrypt=False)

        # Save the decrypted audio
        decrypted_audio_path = "decrypted_audio.wav"
        audio = AudioSegment.from_file(io.BytesIO(decrypted_data), format="wav")
        audio.export(decrypted_audio_path, format="wav")
        print(f"Audio decrypted and saved to {decrypted_audio_path}")

    except Exception as e:
        print(f"Error during decryption: {e}")


# Execution Start
if __name__ == "__main__":
    choice = input("Do you want to (E)ncrypt or (D)ecrypt? ").strip().lower()
    if choice == "e":
        encrypt_audio()
    elif choice == "d":
        decrypt_audio()
    else:
        print("Invalid choice. Please enter 'E' to encrypt or 'D' to decrypt.")


Do you want to (E)ncrypt or (D)ecrypt?  D
Enter the sender's public key:  4
Enter the shared prime number (p):  7
Enter the generator (g):  31


Receiver's public key: 3


Enter the path of the encrypted audio file:  C:\Users\Yash\Downloads\encrypted_audio.wav


Error during decryption: Invalid padding bytes.


In [9]:
import os
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.padding import PKCS7
from cryptography.hazmat.backends import default_backend
from pydub import AudioSegment
import io

# AES Encryption and Decryption
def aes_encrypt_decrypt_audio(audio_data, key, encrypt=True):
    backend = default_backend()
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=backend)
    padder = PKCS7(algorithms.AES.block_size).padder()
    unpadder = PKCS7(algorithms.AES.block_size).unpadder()

    if encrypt:
        padded_data = padder.update(audio_data) + padder.finalize()
        encryptor = cipher.encryptor()
        return encryptor.update(padded_data) + encryptor.finalize()
    else:
        decryptor = cipher.decryptor()
        decrypted_data = decryptor.update(audio_data) + decryptor.finalize()
        return unpadder.update(decrypted_data) + unpadder.finalize()

# Generate RSA Key Pair
def generate_rsa_keys():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    public_key = private_key.public_key()
    return private_key, public_key

# Encrypt the Audio File
def encrypt_audio(public_key):
    try:
        # Generate AES key
        aes_key = os.urandom(32)  # 256-bit key

        # Encrypt AES key with RSA public key
        encrypted_aes_key = public_key.encrypt(
            aes_key,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )

        # Load the audio file
        input_audio_path = input("Enter the path of the audio file to encrypt: ")
        audio = AudioSegment.from_file(input_audio_path)
        audio_bytes = io.BytesIO()
        audio.export(audio_bytes, format="wav")
        audio_data = audio_bytes.getvalue()

        # Encrypt the audio with AES
        encrypted_audio_data = aes_encrypt_decrypt_audio(audio_data, aes_key, encrypt=True)

        # Save encrypted audio and AES key
        with open("encrypted_audio.dat", "wb") as f:
            f.write(encrypted_audio_data)

        with open("encrypted_aes_key.dat", "wb") as f:
            f.write(encrypted_aes_key)

        print("Audio encrypted and saved as 'encrypted_audio.dat'")
        print("AES key encrypted and saved as 'encrypted_aes_key.dat'")

    except Exception as e:
        print(f"Error during encryption: {e}")

# Decrypt the Audio File
def decrypt_audio(private_key):
    try:
        # Load encrypted AES key
        with open("encrypted_aes_key.dat", "rb") as f:
            encrypted_aes_key = f.read()

        # Decrypt AES key with RSA private key
        aes_key = private_key.decrypt(
            encrypted_aes_key,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )

        # Load encrypted audio data
        with open("encrypted_audio.dat", "rb") as f:
            encrypted_audio_data = f.read()

        # Decrypt the audio with AES
        decrypted_audio_data = aes_encrypt_decrypt_audio(encrypted_audio_data, aes_key, encrypt=False)

        # Save the decrypted audio
        with open("decrypted_audio.wav", "wb") as f:
            f.write(decrypted_audio_data)
        print("Audio decrypted and saved as 'decrypted_audio.wav'")

    except Exception as e:
        print(f"Error during decryption: {e}")


# Execution Start
if __name__ == "__main__":
    choice = input("Do you want to (G)enerate RSA keys, (E)ncrypt, or (D)ecrypt? ").strip().lower()
    if choice == "g":
        private_key, public_key = generate_rsa_keys()

        # Save private key
        with open("private_key.pem", "wb") as f:
            f.write(
                private_key.private_bytes(
                    encoding=serialization.Encoding.PEM,
                    format=serialization.PrivateFormat.TraditionalOpenSSL,
                    encryption_algorithm=serialization.NoEncryption()
                )
            )
        print("Private key saved as 'private_key.pem'")

        # Save public key
        with open("public_key.pem", "wb") as f:
            f.write(
                public_key.public_bytes(
                    encoding=serialization.Encoding.PEM,
                    format=serialization.PublicFormat.SubjectPublicKeyInfo
                )
            )
        print("Public key saved as 'public_key.pem'")

    elif choice == "e":
        with open("public_key.pem", "rb") as f:
            public_key = serialization.load_pem_public_key(f.read(), backend=default_backend())
        encrypt_audio(public_key)

    elif choice == "d":
        with open("private_key.pem", "rb") as f:
            private_key = serialization.load_pem_private_key(f.read(), password=None, backend=default_backend())
        decrypt_audio(private_key)
    else:
        print("Invalid choice. Please select 'G', 'E', or 'D'.")


Do you want to (G)enerate RSA keys, (E)ncrypt, or (D)ecrypt?  D


Audio decrypted and saved as 'decrypted_audio.wav'
